# Comparison of context aware masking levels

In [1]:
!pip install beautifulsoup4 lxml -q

Helper Functions

In [2]:
import re
from privacy_pipeline import PrivacyPipeline
from IPython.display import HTML, display
import os
from contextlib import redirect_stdout

async def compare_masking_levels(text, question):
    with open(os.devnull, 'w') as fnull:
        with redirect_stdout(fnull):
            privacy_pipeline_0 = PrivacyPipeline("./test_configs/eng_spacy_context0.yml")
            privacy_pipeline_1 = PrivacyPipeline("./test_configs/eng_spacy_context1.yml")
            privacy_pipeline_2 = PrivacyPipeline("./test_configs/eng_spacy_context2.yml")
            
            result_0 = await privacy_pipeline_0.process_pipeline(text, question)
            result_1 = await privacy_pipeline_1.process_pipeline(text, question)
            result_2 = await privacy_pipeline_2.process_pipeline(text, question)
    
    return [result_0, result_1, result_2]

def create_html_with_entities(text, entity_mappings):
    sorted_entities = sorted(entity_mappings.items(), key=lambda x: len(x[0]), reverse=True)
    def replace_entity(match):
        entity_text = match.group(0)
        for old_entity, new_entity in sorted_entities:
            if old_entity in entity_text:
                return f'<mark class="entity" style="background: #bfe1d9; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">{old_entity} <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">{new_entity}</span></mark>'
        return entity_text

    html_content = re.sub(r'\b(' + '|'.join(re.escape(key) for key in entity_mappings.keys()) + r')\b', replace_entity, text)
    full_html = f"""
    <!DOCTYPE html>
    <html lang="en">
     <body style="font-size: 16px; font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Helvetica, Arial, sans-serif, 'Apple Color Emoji', 'Segoe UI Emoji', 'Segoe UI Symbol'; padding: 4rem 2rem; direction: ltr">
      <figure style="margin-bottom: 6rem">
       <div class="entities" style="line-height: 2.5; direction: ltr">
        {html_content}
       </div>
      </figure>
     </body>
    </html>
    """
    return full_html

async def analyze_context_dependency(text, question):
    results = await compare_masking_levels(text, question)
    final_outputs = []
    for i in range(len(results)):
        
        print(f"==== CONTEXT LEVEL: {i} ====")
        
        print("Question:")
        html_question = create_html_with_entities(question, results[i]["context_mapping"])
        display(HTML(html_question))

        print("Base Knowledge:")
        html_knowledge = create_html_with_entities(text, results[i]["context_mapping"])
        display(HTML(html_knowledge))

        print("Postprocessed LLM response:")
        html_postprocessed = create_html_with_entities(results[i]["final_output"], results[i]["context_mapping"])
        display(HTML(html_postprocessed))
        final_outputs.append(results[i]["final_output"])

    return final_outputs

async def visualize(text, question, final_output, mappings):

    print(f"==== CONTEXT LEVEL: {2} ====")
    
    print("Question:")
    html_question = create_html_with_entities(question, mappings)
    display(HTML(html_question))

    print("Base Knowledge:")
    html_knowledge = create_html_with_entities(text, mappings)
    display(HTML(html_knowledge))

    print("Postprocessed LLM response:")
    html_postprocessed = create_html_with_entities(final_output, mappings)
    display(HTML(html_postprocessed))


/Users/lilianahotsko/Desktop/LLMAnonymizationThesis/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
text = "In 2020, Sarah Mitchell, the CEO of GlobalTech Innovations, announced a major restructuring within the company. This decision, made after months of deliberations, aimed at addressing the impact of the COVID-19 pandemic on the business. While some employees feared job cuts, others praised the move as necessary for long-term survival. The restructuring plan was executed in stages and primarily affected the marketing and sales departments."
question = "What were the main reasons for Sarah Mitchell’s decision to restructure GlobalTech Innovations in 2020?"
await analyze_context_dependency(text, question)


/Users/lilianahotsko/Desktop/LLMAnonymizationThesis/.venv/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


==== CONTEXT LEVEL: 0 ====
Question:


Base Knowledge:


Postprocessed LLM response:


==== CONTEXT LEVEL: 1 ====
Question:


Base Knowledge:


Postprocessed LLM response:


==== CONTEXT LEVEL: 2 ====
Question:


Base Knowledge:


Postprocessed LLM response:


### With final version of prompts in llama

In [12]:
text = "In 2020, Sarah Mitchell, the CEO of GlobalTech Innovations, announced a major restructuring within the company. This decision, made after months of deliberations, aimed at addressing the impact of the COVID-19 pandemic on the business. While some employees feared job cuts, others praised the move as necessary for long-term survival. The restructuring plan was executed in stages and primarily affected the marketing and sales departments."
question = "What were the main reasons for Sarah Mitchell’s decision to restructure GlobalTech Innovations in 2020?"
await analyze_context_dependency(text, question)

==== CONTEXT LEVEL: 0 ====
Question:


Base Knowledge:


Postprocessed LLM response:


==== CONTEXT LEVEL: 1 ====
Question:


Base Knowledge:


Postprocessed LLM response:


==== CONTEXT LEVEL: 2 ====
Question:


Base Knowledge:


Postprocessed LLM response:


### LLM bias on the NAME

In [3]:
from datasets import Dataset

def create_ragas_dataset(question,
                        base_text,
                        ground_truth_response,
                        final_outputs):
    base_texts = []
    base_texts.append(base_text)
    ragas_data = Dataset.from_dict({
        "question": [question, question, question],
        "answer": final_outputs,
        "contexts": [base_texts, base_texts, base_texts],
        "reference": [ground_truth_response, ground_truth_response, ground_truth_response]
    })

    return ragas_data

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
)

def evaluate_rag(ragas_data):
    result = evaluate(
        dataset=ragas_data,
        metrics=[
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall,
            answer_correctness
        ]
    )
    return result



## Example 1 - Indian customer

In [14]:
from providers.openai_provider import OpenAIProvider
from IPython.display import Markdown, display
openai = OpenAIProvider()

text = "Anjali Gupta is the head of business development at a corporation in India. Take into account what values this category of customers have. "
question = "What is the best approach to building rapport with a customer Anjali Gupta?"
ground_truth_response = await openai.query_llm(text+question)
print("Ground truth response: ")
display(Markdown(ground_truth_response))
final_outputs = await analyze_context_dependency(text, question)

ragas_data = create_ragas_dataset(question,
                                        text,
                                        ground_truth_response,
                                        final_outputs)
evaluation = evaluate_rag(ragas_data)

pd_results = evaluation.to_pandas()
df_dropped = pd_results.drop(pd_results.columns[:4], axis=1)
df_dropped


Ground truth response: 


To build rapport with a customer like Anjali Gupta, who is the head of business development at a corporation in India, it's important to understand and align with the values typical of this role and cultural context. Here are some key values and approaches to consider:

### Values:
1. **Professionalism**: Business development leaders value professionalism and competence. Demonstrating expertise in your area and understanding her industry trends is crucial.

2. **Strategic Thinking**: Show an understanding of long-term business goals and provide solutions that contribute to them.

3. **Trust and Reliability**: Building trust is essential. Honesty, transparency, and consistency in your dealings will be valued.

4. **Relationship Building**: Business development roles often involve relationship building. Demonstrate a genuine interest in her role and company.

5. **Innovation and Growth**: Highlight innovative solutions or opportunities for business growth, as these are central to the business development function.

6. **Cultural Understanding**: Cultural sensitivity and awareness can build a deeper connection. Respect Indian business etiquette which often leans towards formality.

### Approach to Building Rapport:
1. **Research and Preparation**: Understand Anjali’s professional background, her company’s business model, and industry trends. Tailor your discussions to align with her interests and company objectives.

2. **Personalization**: Personalize your communication. Reference specific achievements of her company or initiatives she’s led.

3. **Effective Communication**: Be clear, concise, and articulate. Listening actively and acknowledging her points can help in building a connection.

4. **Value Proposition**: Clearly communicate the value you can bring to her business. Use data and case studies to back your claims.

5. **Regular Engagement**: Maintain regular, meaningful interactions. Follow-ups and updates demonstrate your commitment to the relationship.

6. **Cultural Etiquette**: Be mindful of cultural norms, such as addressing her with appropriate titles unless invited to do otherwise, and being punctual and respectful in meetings.

7. **Empathy and Understanding**: Understand her challenges and pressures. Offering solutions that address her pain points can build rapport.

8. **Networking and Introductions**: Facilitate introductions to other professionals or opportunities that could benefit her, indicating a willingness to support her beyond immediate business dealings.

By aligning your approach with these values and strategies, you can effectively build a cooperative and long-lasting rapport with Anjali Gupta.

/Users/lilianahotsko/Desktop/LLMAnonymizationThesis/.venv/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


==== CONTEXT LEVEL: 0 ====
Question:


Base Knowledge:


...

==== CONTEXT LEVEL: 2 ====
Question:


Base Knowledge:


Postprocessed LLM response:


Evaluating: 100%|██████████| 15/15 [00:41<00:00,  2.79s/it]


,faithfulness,answer_relevancy,context_precision,context_recall,answer_correctness
0,0.217391,0.972329,1.0,0.375000,0.408590
1,0.600000,0.956276,1.0,0.466667,0.470989
2,0.500000,0.972329,1.0,1.000000,0.984300


## Example 2 - Western Customer

In [20]:
from providers.openai_provider import OpenAIProvider
from IPython.display import Markdown, display
openai = OpenAIProvider()

text = "John Smith is the head of business development at a corporation in Silicon Valley. Take into account what values this category of customers have. "
question = "What is the best approach to building rapport with a customer John Smith?"
ground_truth_response = await openai.query_llm(text+question)
print("Ground truth response: ")
display(Markdown(ground_truth_response))
final_outputs = await analyze_context_dependency(text, question)

ragas_data = create_ragas_dataset(question,
                                        text,
                                        ground_truth_response,
                                        final_outputs)
evaluation = evaluate_rag(ragas_data)

pd_results = evaluation.to_pandas()
df_dropped = pd_results.drop(pd_results.columns[:4], axis=1)
df_dropped


Ground truth response: 


Building rapport with someone like John Smith, who is the head of business development at a Silicon Valley corporation, requires a thoughtful and strategic approach. Executives in this role often value innovation, efficiency, strategic growth opportunities, and fostering strong professional relationships. Here’s a guide on how to build rapport effectively:

1. **Do Your Homework**: Before any interaction, thoroughly research John’s company and its industry. Understand its business model, recent developments, and challenges. Be aware of John’s professional background and any public contributions he has made, such as speaking engagements or publications.

2. **Be Authentic and Genuine**: Authenticity is crucial. Engage in conversations that are sincere and demonstrate genuine interest in their work and success. Avoid any semblance of flattery or insincerity.

3. **Highlight Shared Values**: Identify and emphasize any shared values or interests, such as a commitment to innovation, sustainability, or diversity in business practices. This can help establish common ground.

4. **Offer Value**: Present insights, ideas, or opportunities that could be valuable to John and his company. This might include market trends, potential collaborations, or technologies that align with their strategic goals.

5. **Listen Actively**: Practice active listening to understand John’s priorities, concerns, and business objectives. This will help tailor your conversation and offer relevant solutions.

6. **Be Technologically Savvy**: Demonstrating an understanding of the latest technology trends and how they could impact or benefit his business shows credibility and relevance, particularly in the fast-moving Silicon Valley environment.

7. **Build Through Networking**: If possible, connect through a mutual acquaintance or attend industry events where John might be present. Leverage professional networks like LinkedIn to facilitate a warm introduction.

8. **Be Respectful of Time**: Acknowledge the value of time and keep communications concise and to the point. Show that you respect his busy schedule by being punctual and prepared for meetings or calls.

9. **Follow-Up Thoughtfully**: After an initial meeting or call, send a follow-up note summarizing key points discussed, expressing gratitude for the opportunity to connect, and suggesting potential next steps or additional value you can provide.

10. **Maintain Integrity**: Always maintain a high degree of integrity in all interactions. Trust is critical in building and maintaining professional relationships, especially at the executive level.

By combining these strategies, you can build a strong rapport with John Smith, demonstrating that you are a trusted partner invested in mutual success.

/Users/lilianahotsko/Desktop/LLMAnonymizationThesis/.venv/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


==== CONTEXT LEVEL: 0 ====
Question:


Base Knowledge:


...

==== CONTEXT LEVEL: 2 ====
Question:


Base Knowledge:


Postprocessed LLM response:


Evaluating: 100%|██████████| 15/15 [00:54<00:00,  3.62s/it]


,faithfulness,answer_relevancy,context_precision,context_recall,answer_correctness
0,0.0,0.909511,1.0,0.153846,0.546293
1,1.0,0.959563,1.0,0.153846,0.407244
2,0.0,0.897696,1.0,0.153846,0.454730


In [21]:
from providers.openai_provider import OpenAIProvider
from IPython.display import Markdown, display
openai = OpenAIProvider()

text = "John Smith is the head of business development at a corporation in Silicon Valley. Take into account what values this category of customers have. "
question = "What is the best approach to building rapport with a customer John Smith?"
ground_truth_response = await openai.query_llm(text+question)
print("Ground truth response: ")
display(Markdown(ground_truth_response))
final_outputs = await analyze_context_dependency(text, question)

ragas_data = create_ragas_dataset(question,
                                        text,
                                        ground_truth_response,
                                        final_outputs)
evaluation = evaluate_rag(ragas_data)

pd_results = evaluation.to_pandas()
df_dropped = pd_results.drop(pd_results.columns[:4], axis=1)
df_dropped


Ground truth response: 


Building rapport with a customer like John Smith, who is the head of business development at a Silicon Valley corporation, involves understanding his professional values and the business environment he operates in. People in such roles typically appreciate efficiency, innovation, and strong results-oriented relationships. Here are some tailored strategies to effectively build rapport with him:

1. **Understand Industry Trends**: Demonstrate knowledge about the latest trends in technology and business development. This shows that you are engaged and knowledgeable about the environment he operates in.

2. **Focus on Value**: Clearly articulate the value your product or service brings to his business. Be prepared to discuss ROI, growth potential, or problem-solving capabilities that align with his company's goals.

3. **Be Efficient and Respectful of Time**: Silicon Valley professionals often have packed schedules. Be concise in your communication and ensure meetings or proposals are efficient and to the point.

4. **Demonstrate Credibility**: Leverage any relevant experience or partnerships you have within the tech industry. Sharing success stories or case studies can establish your credibility quickly.

5. **Engage in Solution-Oriented Conversations**: Focus discussions on solutions that address his business challenges. Be prepared with insights and examples of how your offering can distinctively solve problems.

6. **Build a Personal Connection**: While maintaining professionalism, find common ground on shared interests or mutual connections that extend beyond business. This could be about industry-related hobbies or broader Silicon Valley-related events.

7. **Highlight Innovation**: Silicon Valley values innovation, so emphasize how your product or service is unique or how it leverages cutting-edge technology.

8. **Follow Up Thoughtfully**: After any interaction, a thoughtful follow-up email or message can help reinforce your conversation. Recap key points and outline next steps to keep the momentum going.

9. **Show Long-term Dedication**: Express your interest in building a long-lasting partnership rather than a transactional relationship. This indicates your commitment to supporting his organization's long-term success.

By aligning your approach with these strategies, you can effectively build rapport with John Smith and foster a strong professional relationship.

/Users/lilianahotsko/Desktop/LLMAnonymizationThesis/.venv/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


==== CONTEXT LEVEL: 0 ====
Question:


Base Knowledge:


...

==== CONTEXT LEVEL: 2 ====
Question:


Base Knowledge:


Postprocessed LLM response:


Evaluating: 100%|██████████| 15/15 [00:52<00:00,  3.51s/it]


,faithfulness,answer_relevancy,context_precision,context_recall,answer_correctness
0,0.0,0.959010,1.0,0.181818,0.474078
1,1.0,0.953199,1.0,0.250000,0.987251
2,0.0,0.875217,1.0,0.181818,0.420364


## Example 3 - Japanese Customer

In [19]:
from providers.openai_provider import OpenAIProvider
from IPython.display import Markdown, display
openai = OpenAIProvider()

text = "Hiroshi Tanaka is the head of business development at a corporation in Japan. Take into account what values this category of customers have. "
question = "What is the best approach to building rapport with a customer Hiroshi Tanaka?"
ground_truth_response = await openai.query_llm(text+question)
print("Ground truth response: ")
display(Markdown(ground_truth_response))
final_outputs = await analyze_context_dependency(text, question)

ragas_data = create_ragas_dataset(question,
                                        text,
                                        ground_truth_response,
                                        final_outputs)
evaluation = evaluate_rag(ragas_data)

pd_results = evaluation.to_pandas()
df_dropped = pd_results.drop(pd_results.columns[:4], axis=1)
df_dropped


Ground truth response: 


Building rapport with Hiroshi Tanaka, the head of business development at a corporation in Japan, requires an approach that aligns with the cultural and business values typically important in Japan. Here are key strategies to consider:

1. **Demonstrate Respect and Politeness:**
   - Use formal titles and surnames unless instructed otherwise. For example, address him as "Tanaka-san."
   - Show respect for his position and experience within the company.

2. **Build a Trusting Relationship:**
   - Establish credibility by being transparent, honest, and reliable.
   - Follow through on commitments and deliver what you promise.

3. **Understand Hierarchical Structures:**
   - Be aware of the corporate hierarchy and show appropriate respect to those in senior positions.
   - Understand decision-making processes, which might be more group and consensus-oriented.

4. **Patience and Long-term Focus:**
   - Business relationships in Japan often develop over time. Exhibit patience and focus on long-term partnership rather than quick wins.
   - Be prepared for multiple meetings to build trust and consensus.

5. **Attention to Detail:**
   - Ensure that any presentation or documentation provided is thorough and free of errors.
   - Pay attention to quality and detail, as these are highly valued.

6. **Cultural Awareness:**
   - Be aware of basic Japanese customs and etiquette. For example, offer your business card with both hands and accept his in the same manner while taking a moment to read it.
   - Understand the importance of harmony (WA) and avoid confrontational or aggressive behavior.

7. **Adapt to Communication Styles:**
   - Listen actively and be attuned to non-verbal cues.
   - Japanese business culture often values indirect communication; pay attention to what may not be explicitly stated.

8. **Present Sound Business Knowledge:**
   - Demonstrate a thorough understanding of the market and industry.
   - Offer insights and solutions that are tailored to his company’s needs and challenges.

By respecting these values and adapting to cultural nuances, you can effectively build rapport and foster a productive relationship with Hiroshi Tanaka.

/Users/lilianahotsko/Desktop/LLMAnonymizationThesis/.venv/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


==== CONTEXT LEVEL: 0 ====
Question:


Base Knowledge:


...

==== CONTEXT LEVEL: 2 ====
Question:


Base Knowledge:


Postprocessed LLM response:


Evaluating: 100%|██████████| 15/15 [00:56<00:00,  3.75s/it]


,faithfulness,answer_relevancy,context_precision,context_recall,answer_correctness
0,0.739130,0.951346,1.0,0.5,0.402583
1,1.000000,0.935949,1.0,0.1,0.989066
2,0.071429,0.941235,1.0,0.4,0.459148
